In [1]:
from openff.evaluator.properties import Density

In [2]:
schema = Density.default_simulation_schema()

In [3]:
for s in schema.workflow_schema.protocol_schemas:
    print(s.id,"\n",s.type,"\n",s.inputs,"\n")

build_coordinates 
 BuildCoordinatesPackmol 
 {'.allow_merging': True, '.max_molecules': 1000, '.count_exact_amount': True, '.mass_density': <Quantity(0.95, 'gram / milliliter')>, '.box_aspect_ratio': [1.0, 1.0, 1.0], '.substance': <ProtocolPath full_path=global.substance>, '.tolerance': <Quantity(2.0, 'angstrom')>, '.verbose_packmol': False, '.retain_packmol_files': False} 

assign_parameters 
 BaseBuildSystem 
 {'.allow_merging': True, '.force_field_path': <ProtocolPath full_path=global.force_field_path>, '.coordinate_file_path': <ProtocolPath full_path=build_coordinates.coordinate_file_path>, '.substance': <ProtocolPath full_path=build_coordinates.output_substance>} 

energy_minimisation 
 OpenMMEnergyMinimisation 
 {'.allow_merging': True, '.input_coordinate_file': <ProtocolPath full_path=build_coordinates.coordinate_file_path>, '.parameterized_system': <ProtocolPath full_path=assign_parameters.parameterized_system>, '.tolerance': <Quantity(10.0, 'kilojoule / mole')>, '.max_iterati

In [4]:
density_schema = Density.default_simulation_schema(n_molecules=256)

In [5]:

from openff.evaluator.client import RequestOptions

# Create an options object which defines how the data set should be estimated.
estimation_options = RequestOptions()
# Specify that we only wish to use molecular simulation to estimate the data set.
estimation_options.calculation_layers = ["SimulationLayer"]

# Add our custom schemas, specifying that the should be used by the 'SimulationLayer'
estimation_options.add_schema("SimulationLayer", "Density", density_schema)



In [6]:
import os

from openff.evaluator.backends import ComputeResources
from openff.evaluator.backends.dask import DaskLocalCluster

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

calculation_backend = DaskLocalCluster(
    number_of_workers=1,
    resources_per_worker=ComputeResources(
        number_of_threads=1,
        number_of_gpus=1,
        preferred_gpu_toolkit=ComputeResources.GPUToolkit.CUDA,
    ),
)
calculation_backend.start()


In [7]:
from openff.evaluator.datasets import PhysicalPropertyDataSet
data_set = PhysicalPropertyDataSet()


In [8]:
from openff.units import unit

from openff.evaluator.thermodynamics import ThermodynamicState

thermodynamic_state = ThermodynamicState(
    temperature=298.15 * unit.kelvin, pressure=1.0 * unit.atmosphere
)

In [9]:
from openff.evaluator.substances import Substance

ethanol = Substance.from_components("CCO")

In [10]:
from openff.evaluator.datasets import PropertyPhase
from openff.evaluator.properties import Density
ethanol_density = Density(
    thermodynamic_state=thermodynamic_state,
    phase=PropertyPhase.Liquid,
    substance=ethanol,
    value=1.0 * unit.gram / unit.milliliter,
)

In [11]:
data_set.add_properties(ethanol_density)

In [12]:
data_set.to_pandas().head()

,Id,Temperature (K),Pressure (kPa),Phase,N Components,Component 1,Role 1,Mole Fraction 1,Exact Amount 1,Density Value (g / ml),Density Uncertainty (g / ml),Source
0,69872be6517f4e22bdab4903c2c292ef,298.15,101.325,Liquid,1,CCO,Solvent,1.0,None,1.0,None,None


In [13]:
from openff.evaluator.server import EvaluatorServer

evaluator_server = EvaluatorServer(calculation_backend=calculation_backend)
evaluator_server.start(asynchronous=True)

In [14]:
from openff.evaluator.forcefield import SmirnoffForceFieldSource

force_field_path = "openff-1.0.0.offxml"
force_field_source = SmirnoffForceFieldSource.from_path(force_field_path)

In [15]:
from openff.evaluator.client import EvaluatorClient

evaluator_client = EvaluatorClient()

request, exception = evaluator_client.request_estimate(
    property_set=data_set,
    force_field_source=force_field_source,
    options=estimation_options,
)

assert exception is None

In [16]:
# Wait for the results.
results, exception = request.results(synchronous=True, polling_interval=30)
assert exception is None


In [17]:


print(len(results.queued_properties))

print(len(results.estimated_properties))

print(len(results.unsuccessful_properties))
print(len(results.exceptions))



0
0
1
1


In [18]:
results.unsuccessful_properties

TypeError: unhashable type: 'UndefinedAttribute'

In [19]:

from openff.evaluator.workflow.schemas import WorkflowSchema
# Create the schema object.
schema = WorkflowSchema()
# Add the individual protocol's schema representations to the workflow schema.
schema.protocol_schemas = [density_schema]

In [20]:
schema.get_attributes()

['protocol_schemas',
 'protocol_replicators',
 'final_value_source',
 'outputs_to_store']

In [21]:
from openff.evaluator.workflow import Workflow
from openff.evaluator.substances import Substance
from openff.evaluator.substances import Component
from openff.evaluator.substances import MoleFraction
smiles = "CCO"
substance = Substance()
substance.add_component(Component(smiles=smiles), MoleFraction(1.0))
metadata = {"substance": substance}
density_schema = Density.default_simulation_schema()
density_workflow = Workflow.from_schema(density_schema, metadata)

AttributeError: 'SimulationSchema' object has no attribute 'final_value_source'

In [ ]:
# Step 9: Create the workflow from schema
workflow = Workflow.from_schema(density_schema, metadata=metadata)

# Step 10: Execute the workflow
workflow.execute()


AttributeError: 'SimulationSchema' object has no attribute 'final_value_source'

In [ ]:
from openff.evaluator.client import RequestOptions

# Create the options which evaluator should use.
evaluator_options = RequestOptions()
# Choose which calculation layers to make available.
evaluator_options.calculation_layers = ["SimulationLayer"]

# Reduce the default number of molecules
from openff.evaluator.properties import Density, EnthalpyOfVaporization

density_schema = Density.default_simulation_schema(n_molecules=256)
h_vap_schema = EnthalpyOfVaporization.default_simulation_schema(n_molecules=256)

evaluator_options.add_schema("SimulationLayer", "Density", density_schema)

import os

# Launch the calculation backend which will distribute any calculations.
from openff.evaluator.backends import ComputeResources
from openff.evaluator.backends.dask import DaskLocalCluster

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

calculation_backend = DaskLocalCluster(
    number_of_workers=1,
    resources_per_worker=ComputeResources(
        number_of_threads=1,
        number_of_gpus=1,
        preferred_gpu_toolkit=ComputeResources.GPUToolkit.CUDA,
    ),
)
calculation_backend.start()

# Launch the server object which will listen for estimation requests and schedule any
# required calculations.
from openff.evaluator.server import EvaluatorServer

evaluator_server = EvaluatorServer(calculation_backend=calculation_backend)
evaluator_server.start(asynchronous=True)

/localhome/cschiebroek/.conda/envs/openff-evaluator/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45849 instead
  warnings.warn(


OSError: [Errno 98] Address already in use

In [ ]:
from openff.evaluator import unit
from openff.evaluator.backends.dask import DaskLocalCluster
from openff.evaluator.client import RequestOptions, EvaluatorClient
from openff.evaluator.datasets import PhysicalPropertyDataSet
from openff.evaluator.datasets import PhysicalProperty
from openff.evaluator.forcefield import ParameterGradient
from openff.evaluator.layers.simulation import SimulationLayer, SimulationSchema

from openff.evaluator.protocols.groups import ProtocolGroup
from openff.evaluator.substances import Substance
from openff.evaluator.thermodynamics import ThermodynamicState

# Define the substance (e.g., ethanol)
smiles = "CCO"
substance = Substance()
substance.add_component(Component(smiles=smiles), MoleFraction(1.0))

# Define the thermodynamic state (e.g., 298.15 K and 1 atm)
thermodynamic_state = ThermodynamicState(
    temperature=298.15 * unit.kelvin,
    pressure=1.0 * unit.atmosphere
)

# Define the dataset
dataset = PhysicalPropertyDataSet()
dataset.add_properties(
    DataSetProperty(
        thermodynamic_state=thermodynamic_state,
        substance=substance,
        property_type="Density"
    )
)

# Set up the evaluator server and client
server = PropertyEstimatorServer()
cluster = DaskLocalCluster()
client = EvaluatorClient(server)

# Define the request options
options = RequestOptions()
options.calculation_layers = [
    SimulationLayer,  # Run MD simulations
]

# Request the estimation of the density
request = client.request_estimate(
    property_set=dataset,
    options=options
)

# Wait until the estimate has completed
results = request.results()

# Check for any errors
if results.errors:
    raise RuntimeError("The estimation failed.")

# Extract the estimated density
estimated_density = results.estimated_properties[0].value.to(unit.gram / unit.milliliter)

print(f"Estimated density: {estimated_density}")

# Now perform a 5 ns production simulation
production_protocol = ProtocolGroup()
production_protocol.add_protocol(
    SimulationLayer(
        production_time=5.0 * unit.nanoseconds,
        thermodynamic_state=thermodynamic_state,
        substance=substance
    ),
    protocol_id="production_simulation"
)

# Execute the production simulation
production_request = client.request_estimate(
    property_set=dataset,
    options=RequestOptions(calculation_layers=[production_protocol])
)

# Wait until the production simulation has completed
production_results = production_request.results()

# Check for any errors
if production_results.errors:
    raise RuntimeError("The production simulation failed.")

print("Production simulation completed successfully.")


ValueError: The required value attribute has not been set.

In [ ]:
import pathlib

from openff.evaluator.datasets import PhysicalPropertyDataSet

data_set_path = "filtered_data_set.json"


# If you have not yet completed that tutorial or do not have the data set file
# available, this tutorial will use a copies provided by the framework

if not pathlib.Path(data_set_path).exists():
    from openff.evaluator.utils import get_data_filename

    data_set_path = get_data_filename("tutorials/tutorial01/filtered_data_set.json")

data_set = PhysicalPropertyDataSet.from_json(data_set_path)
data_set

<PhysicalPropertyDataSet n_properties=4 n_substances=2 n_sources=4>

In [ ]:
import numpy as np
data_set.to_pandas().head()
#keep only the first entry
data_set_pd = data_set.to_pandas().head(1)
data_set_pd
#remove columns EnthalpyOfVaporization and ExcessMolarVolume
data_set_pd = data_set_pd.drop(columns=['EnthalpyOfVaporization Value (kJ / mol)','EnthalpyOfVaporization Uncertainty (kJ / mol)'])
#set Density Value and uncertainty to None
data_set_pd['Density Value (g / ml)'] = np.nan
data_set_pd['Density Uncertainty (g / ml)'] = np.nan
data_set_pd['Source'] = np.nan
data_set_pd


,Id,Temperature (K),Pressure (kPa),Phase,N Components,Component 1,Role 1,Mole Fraction 1,Exact Amount 1,Density Value (g / ml),Density Uncertainty (g / ml),Source
0,ae520b7a60084133bba8f8ee6eb8d8b5,298.15,101.325,Liquid,1,CC(C)O,Solvent,1.0,None,NaN,NaN,NaN


In [ ]:
data_set = PhysicalPropertyDataSet.from_pandas(data_set_pd)
data_set

<PhysicalPropertyDataSet n_properties=0 n_substances=0 n_sources=0>

In [ ]:
from openff.evaluator.properties import Density, EnthalpyOfVaporization

density_schema = Density.default_simulation_schema(n_molecules=256)

In [ ]:
from openff.evaluator.client import RequestOptions

# Create an options object which defines how the data set should be estimated.
estimation_options = RequestOptions()
estimation_options.calculation_layers = ["SimulationLayer"]
estimation_options.add_schema("SimulationLayer", "Density", density_schema)


In [ ]:
import os

from openff.evaluator.backends import ComputeResources
from openff.evaluator.backends.dask import DaskLocalCluster

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

calculation_backend = DaskLocalCluster(
    number_of_workers=1,
    resources_per_worker=ComputeResources(
        number_of_threads=1,
        number_of_gpus=1,
        preferred_gpu_toolkit=ComputeResources.GPUToolkit.CUDA,
    ),
)
calculation_backend.start()

/localhome/cschiebroek/.conda/envs/openff-evaluator/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33593 instead
  warnings.warn(


In [ ]:
from openff.evaluator.server import EvaluatorServer

evaluator_server = EvaluatorServer(calculation_backend=calculation_backend)
#stop the server
evaluator_server.start(asynchronous=True)
evaluator_server.stop()

OSError: [Errno 98] Address already in use

In [ ]:
from openff.evaluator.client import EvaluatorClient

evaluator_client = EvaluatorClient()

request, exception = evaluator_client.request_estimate(
    property_set=data_set,
    force_field_source=force_field_source,
    options=estimation_options,
)

assert exception is None

In [ ]:
data_set.to_pandas().head()

""


In [ ]:
# Wait for the results.
results, exception = request.results(synchronous=True, polling_interval=30)
assert exception is None

In [ ]:
print(len(results.queued_properties))

print(len(results.estimated_properties))

print(len(results.unsuccessful_properties))
print(len(results.exceptions))

0
0
4
4
